In [1]:
from pyspark.sql import SparkSession

SyntaxError: invalid syntax (Temp/ipykernel_17436/1315092646.py, line 1)

In [10]:
import json
with open("test.jsonl") as f:
    train_data = [json.loads(s) for s in f.readlines()]

JSONDecodeError: Expecting value: line 2 column 1 (char 1)

In [14]:
f=open('data.json',"r")
train_data=[]
for line in f.readlines():
    data=json.loads(line)
    train_data.append(data)

In [5]:
import pandas as pd
from area_calculation import *
df = pd.read_json("data.json", lines=True)
print(df.shape)
df.head()

(14800, 5)


,type,width,height,base,radius
0,rectangle,5.0,10.5,NaN,NaN
1,triangle,NaN,3.0,2.0,NaN
2,circle,NaN,NaN,NaN,4.0
3,rectangle,5.0,5.0,NaN,NaN
4,rectangle,5.0,10.0,NaN,NaN


In [14]:
import time
import pandas as pd
from area_calculation import *
s_t = time.time()
df = pd.read_json("data.json", lines=True)
def calculate_area(row):
    shape_type = row['type']
    if shape_type == 'rectangle':
        return rectangle_area(row['width'],row['height'])
    elif shape_type == 'triangle':
        return triangle_area(row['base'],row['height'])
    elif shape_type == 'circle':
        return circle_area(row['radius'])
    else:
        return None  # if the shape type is unknown, return None
df['area'] = df.apply(calculate_area, axis=1)
df = df.dropna(subset=['area'])

    # Print the total area of all shapes
print(f"Total area of all shapes: {df['area'].sum()}")
e_t = time.time()
print('hh',e_t - s_t)

Total area of all shapes: 476432.28509251575
hh 0.20201349258422852


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType

spark = SparkSession.builder.appName("CalculateArea").getOrCreate()

def calculate_area(shape_type, dimension1, dimension2=None):
    if shape_type == "rectangle":
        if dimension1 and dimension2:
            return dimension1 * dimension2
    elif shape_type == "triangle":
        if dimension1 and dimension2:
            return 0.5 * dimension1 * dimension2
    elif shape_type == "circle":
        if dimension1:
            return 3.14159 * dimension1 * dimension1
    else:
        return None  # return None if shape type is unknown or dimensions are missing

udf_calculate_area = udf(calculate_area, FloatType())

# Load data
df = spark.read.json("data.json")

# Add area column
df = df.withColumn("area", udf_calculate_area(df.type, df.width, df.height))

# Drop rows where area is None (these are the rows with invalid shape type or missing dimensions)
df = df.na.drop(subset=["area"])

# Persist the DataFrame
df.persist()

# Calculate total area
total_area = df.agg({"area": "sum"}).collect()[0][0]

print(f"Total area: {total_area}")


Exception: Java gateway process exited before sending its port number

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col, lit
import math

# Start a Spark Session
spark = SparkSession.builder.getOrCreate()

# Load JSON data
df = spark.read.json("data.json")

# Define function to calculate area
def calculate_area(shape_type, dimension1, dimension2=None):
    return when(shape_type == "rectangle", dimension1 * dimension2).otherwise(
        when(shape_type == "circle", math.pi * (dimension1/2)**2).otherwise(
            when(shape_type == "triangle", 0.5 * dimension1 * dimension2).otherwise(lit(0))))

# Add area column
df = df.withColumn("area", calculate_area(col("type"), col("width").otherwise(col("radius")).otherwise(col("base")), col("height")))

# Calculate total area
total_area = df.selectExpr("sum(area)").collect()[0][0]

Exception: Java gateway process exited before sending its port number

In [2]:
import os
os.environ["JAVA_HOME"] = 'E:\java\jdk-19'